We will build our model as a real time inference as a real time inference. When it comes to anolmaly detection , finding out anomalies immediately is of importance.

In [1]:
!pip install s3fs
import pandas as pd
import s3fs

  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.


In [2]:
df= pd.read_csv('edited_Data.csv')

# Split our training and testing datasets

In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

print(train_df.shape, test_df.shape)
train_df.head()

(16558, 61) (1840, 61)


,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x51,x52,x54,x55,x56,x57,x58,x59,x60,y.1
4860,0,-1.088270,1.859843,0.464197,-1.223490,-0.557678,-1.573478,-0.349600,-0.419771,0.520565,...,0.121987,-0.000540,0.967422,0.111115,-0.296084,-1.007830,0.627727,0.107659,0.284658,0
6306,0,0.482483,1.274657,0.513831,-1.653345,-0.708020,-1.211150,0.137834,-0.010658,-1.750726,...,0.121987,1.163074,-0.285351,0.808122,0.878549,1.287519,0.337987,-0.618856,-0.395693,0
1983,0,-0.619647,-1.135704,-0.386512,-1.019537,0.329832,-0.285538,0.867519,-0.149109,0.200114,...,0.121987,-1.325093,0.468599,-2.172190,-1.396331,-1.270146,0.163328,2.490290,1.407710,0
1056,0,-1.242100,-0.039832,-0.208114,1.333124,-0.552576,-0.174087,-0.249530,0.125648,-0.624408,...,0.121987,-0.496931,1.025141,-0.624945,-0.775525,-0.601213,0.381108,1.613248,1.017332,0
5847,0,0.871601,0.132907,0.226597,-1.163488,-0.344645,1.541768,0.137834,-0.010658,0.016516,...,-0.017711,1.437195,0.158081,1.340231,1.535447,1.130129,0.961458,-0.374764,0.560444,0


# Write the training and testing datasets to S3

In [4]:
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()

print(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker-eu-north-1-730335315002


In [5]:
# Write the files locally
import os
import pandas as pd

# Create the "data" directory
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
# Writing DataFrames to CSV files
train_df.to_csv(os.path.join(data_dir, "train.csv"), index=False)
test_df.to_csv(os.path.join(data_dir, "test.csv"), index=False)

print('CSV files written successfully.')


CSV files written successfully.


In [6]:
# Send the files to S3
train_path = session.upload_data(
    path="../data/train.csv", bucket=bucket, key_prefix="sagemaker/anomaly"
)

test_path = session.upload_data(
    path="../data/test.csv", bucket=bucket, key_prefix="sagemaker/anomaly"
)

print(f"Train path: {train_path}")
print(f"Test path: {test_path}")

Train path: s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/train.csv
Test path: s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/test.csv


# Load the train data

In [7]:
import pandas as pd

train_df = pd.read_csv(
    "s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/train.csv",nrows=100)

test_df = pd.read_csv("s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/test.csv") 


print(train_df.shape)
train_df.head()

(100, 61)


,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x51,x52,x54,x55,x56,x57,x58,x59,x60,y.1
0,0,-1.088270,1.859843,0.464197,-1.223490,-0.557678,-1.573478,-0.349600,-0.419771,0.520565,...,0.121987,-0.000540,0.967422,0.111115,-0.296084,-1.007830,0.627727,0.107659,0.284658,0
1,0,0.482483,1.274657,0.513831,-1.653345,-0.708020,-1.211150,0.137834,-0.010658,-1.750726,...,0.121987,1.163074,-0.285351,0.808122,0.878549,1.287519,0.337987,-0.618856,-0.395693,0
2,0,-0.619647,-1.135704,-0.386512,-1.019537,0.329832,-0.285538,0.867519,-0.149109,0.200114,...,0.121987,-1.325093,0.468599,-2.172190,-1.396331,-1.270146,0.163328,2.490290,1.407710,0
3,0,-1.242100,-0.039832,-0.208114,1.333124,-0.552576,-0.174087,-0.249530,0.125648,-0.624408,...,0.121987,-0.496931,1.025141,-0.624945,-0.775525,-0.601213,0.381108,1.613248,1.017332,0
4,0,0.871601,0.132907,0.226597,-1.163488,-0.344645,1.541768,0.137834,-0.010658,0.016516,...,-0.017711,1.437195,0.158081,1.340231,1.535447,1.130129,0.961458,-0.374764,0.560444,0


# Split Features and Response

In [8]:
cont_cols = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9",
             "x10","x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
             "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30", 
             "x31", "x32", "x33", "x34", "x35", "x36", "x37", "x38", "x39", "x40",   
             "x41", "x42", "x43", "x44", "x45", "x46", "x47", "x48", "x49", "x50",
             "x51", "x52", "x54", "x55", "x56", "x57", "x58", "x59", "x60"]


In [9]:
# Split X(features) and y(response)
X_train = train_df.drop("y", axis=1)
y_train = train_df["y"]

X_test = test_df.drop("y", axis=1)
y_test = test_df["y"]

# Data Transformations

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [11]:
sc = StandardScaler()
# Column transformer to apply transformations on both categorical and continuous columns
ct = ColumnTransformer([
    ("Scaling", sc, cont_cols)
])

In [12]:
# Random Forest Model as discuseed in the notebook
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

# Pipeline to combine feature engineering and ML model

In [13]:
# Sklearn pipeline
from sklearn.pipeline import Pipeline

pipeline_rfc_model = Pipeline([
    ("Data Transformations", ct),
    ("Random Forest Model", rfc)
])

## Fit the Pipeline Model locally
##### We run it locally to ensure there are no bugs in the code!
##### For this "test" purpose we can just run it on a smaller subset of the data

In [14]:
# Fit the model locally on a smaller subset of data
pipeline_rfc_model.fit(X_train, y_train)

Pipeline(steps=[('Data Transformations',
                 ColumnTransformer(transformers=[('Scaling', StandardScaler(),
                                                  ['x1', 'x2', 'x3', 'x4', 'x5',
                                                   'x6', 'x7', 'x8', 'x9',
                                                   'x10', 'x11', 'x12', 'x13',
                                                   'x14', 'x15', 'x16', 'x17',
                                                   'x18', 'x19', 'x20', 'x21',
                                                   'x22', 'x23', 'x24', 'x25',
                                                   'x26', 'x27', 'x28', 'x29',
                                                   'x30', ...])])),
                ('Random Forest Model', RandomForestClassifier())])

In [15]:
# Check the accuracy on training data
train_accuracy = pipeline_rfc_model.score(X_train, y_train)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Check the accuracy on test data
test_accuracy = pipeline_rfc_model.score(X_test, y_test)
print(f"Testing Accuracy: {test_accuracy:.4f}")

Training Accuracy: 1.0000
Testing Accuracy: 0.9935


## Fit the Pipeline Model on Sagemaker!
Since the model is free from bugs, we can train it on the full dataset.

In [16]:
%%writefile train.py

import argparse
import os
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

model_file_name = "pipeline_model.joblib"

# Main function
def main():
    # Arguments
    parser = argparse.ArgumentParser()
    
    # Inbuilt Arguments: https://github.com/aws/sagemaker-containers#id11
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    
    # Custom Arguments
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--min_samples_split", type=float, default=0.05)
    parser.add_argument("--criterion", type=str, default="gini")
    
    args, _ = parser.parse_known_args()
    
    
    # Load data
    train_df = pd.read_csv("s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/train.csv") 
    test_df = pd.read_csv("s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/test.csv")

    # Define the columns
    cont_cols = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9",
             "x10","x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
             "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30", 
             "x31", "x32", "x33", "x34", "x35", "x36", "x37", "x38", "x39", "x40",   
             "x41", "x42", "x43", "x44", "x45", "x46", "x47", "x48", "x49", "x50",
             "x51", "x52", "x54", "x55", "x56", "x57", "x58", "x59", "x60"]

    # Split X(features) and y(response)
    X_train = train_df.drop("y", axis=1)
    y_train = train_df["y"]

    X_test = test_df.drop("y", axis=1)
    y_test = test_df["y"]

    # Scale the continuous columns
    sc = StandardScaler()

    # Column transformer to apply transformations on both categorical and continuous columns
    ct = ColumnTransformer([
        ("Scaling", sc, cont_cols)
    ])
    
    # Random Forest Model
    rfc = RandomForestClassifier(n_estimators=args.n_estimators, 
                                 min_samples_split=args.min_samples_split,
                                 criterion=args.criterion)

    # Sklearn pipeline
    pipeline_rfc_model = Pipeline([
        ("Data Transformations", ct),
        ("Random Forest Model", rfc)
    ])

    # Fit the model locally on a smaller subset of data
    pipeline_rfc_model.fit(X_train, y_train)

    # Check the accuracy on training data
    train_accuracy = pipeline_rfc_model.score(X_train, y_train)
    print(f"Training Accuracy: {train_accuracy:.4f}")

    # Check the accuracy on test data
    test_accuracy = pipeline_rfc_model.score(X_test, y_test)
    print(f"Testing Accuracy: {test_accuracy:.4f}")

    # Save the model
    model_save_path = os.path.join(args.model_dir, model_file_name)
    joblib.dump(pipeline_rfc_model, model_save_path)
    print(f"Model saved at {model_save_path}")

# Run the main function when the script runs
if __name__ == "__main__":
    main()

Overwriting train.py


In [17]:
%%writefile requirements.txt
pandas
scikit-learn
fsspec
s3fs

Overwriting requirements.txt


In [18]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

sklearn_estimator = SKLearn(
    base_job_name="rfc-pipeline-run",
    framework_version="1.0-1",
    entry_point="train.py",
    dependencies=["requirements.txt"],
    hyperparameters={
        "n_estimators": 16,
        "min_samples_leaf": 7,
        "max_depth": 14,
        "criterion": "gini"
    },
    instance_count=1,
    instance_type="ml.m5.large",
    use_spot_instances=True,
    max_wait=600,
    max_run=600,
    role=get_execution_role(),
)

# Launch Training job
sklearn_estimator.fit()

INFO:sagemaker:Creating training-job with name: rfc-pipeline-run-2024-03-28-11-25-41-602


2024-03-28 11:25:42 Starting - Starting the training job...
2024-03-28 11:25:58 Starting - Preparing the instances for training...
2024-03-28 11:26:24 Downloading - Downloading input data...
2024-03-28 11:26:44 Downloading - Downloading the training image...
2024-03-28 11:27:40 Training - Training image download completed. Training in progress..2024-03-28 11:27:44,434 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-03-28 11:27:44,436 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-03-28 11:27:44,439 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-03-28 11:27:44,453 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-03-28 11:27:44,676 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 k

In [19]:
import boto3
sm_client = boto3.client("sagemaker")

training_job_name = sklearn_estimator.latest_training_job.name

model_artifact = sm_client.describe_training_job(
    TrainingJobName=training_job_name
)["ModelArtifacts"]["S3ModelArtifacts"]

print(f"Training job name: {training_job_name}")
print(f"Model storage location: {model_artifact}")


Training job name: rfc-pipeline-run-2024-03-28-11-25-41-602
Model storage location: s3://sagemaker-eu-north-1-730335315002/rfc-pipeline-run-2024-03-28-11-25-41-602/output/model.tar.gz


# Hyperparameter Tuning

In [21]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter, CategoricalParameter

# Define exploration boundaries
hyperparameter_ranges = {
    "n_estimators": IntegerParameter(10, 30),
    "min_samples_leaf": IntegerParameter(5, 15),
    "max_depth": IntegerParameter(10, 25),
    "criterion": CategoricalParameter(["gini", "entropy"])
}

# Create optimizer
optimizer = HyperparameterTuner(
    base_tuning_job_name="rfc-pipeline-tuner",
    estimator=sklearn_estimator,
    hyperparameter_ranges=hyperparameter_ranges,
    objective_type="Maximize",
    objective_metric_name="test-accuracy",
    metric_definitions=[
        {"Name": "train-accuracy", "Regex": "Training Accuracy: ([0-9.]+).*$"},
        {"Name": "test-accuracy", "Regex": "Testing Accuracy: ([0-9.]+).*$"}
    ],
    max_jobs=10,
    max_parallel_jobs=2,
)

# Launch Optimizer job
optimizer.fit()

INFO:sagemaker:Creating hyperparameter tuning job with name: rfc-pipeline-tuner-240328-1134


...........................................................................................................................................................................................................!


In [22]:
# Analyse tuning results
results = optimizer.analytics().dataframe()

results.sort_values("FinalObjectiveValue", ascending=False).head()

,criterion,max_depth,min_samples_leaf,n_estimators,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,"""entropy""",22.0,14.0,29.0,rfc-pipeline-tuner-240328-1134-010-ae30df28,Completed,0.9951,2024-03-28 11:49:06+00:00,2024-03-28 11:51:08+00:00,122.0
1,"""entropy""",20.0,6.0,15.0,rfc-pipeline-tuner-240328-1134-009-054a5057,Completed,0.9951,2024-03-28 11:48:25+00:00,2024-03-28 11:50:37+00:00,132.0
2,"""entropy""",22.0,11.0,29.0,rfc-pipeline-tuner-240328-1134-008-080ac24a,Completed,0.9951,2024-03-28 11:45:04+00:00,2024-03-28 11:47:11+00:00,127.0
3,"""entropy""",16.0,7.0,16.0,rfc-pipeline-tuner-240328-1134-007-43a0085b,Completed,0.9951,2024-03-28 11:45:02+00:00,2024-03-28 11:47:04+00:00,122.0
4,"""entropy""",13.0,7.0,29.0,rfc-pipeline-tuner-240328-1134-006-e8158bde,Completed,0.9951,2024-03-28 11:41:44+00:00,2024-03-28 11:43:56+00:00,132.0


### Create the inference script
##### Since the model has been trained with good accuracy we can deploy it.

In [23]:
%%writefile serve.py

import os
import joblib
import pandas as pd

def model_fn(model_dir):
    """Load and return the model"""
    model_file_name = "pipeline_model.joblib"
    pipeline_model = joblib.load(os.path.join(model_dir, model_file_name))
    
    return pipeline_model

def input_fn(request_body, request_content_type):
    """Process the input json data and return the processed data.
    You can also add any input data pre-processing in this function
    """
    if request_content_type == "application/json":
        input_object = pd.read_json(request_body, lines=True)
        
        return input_object
    else:
        raise ValueError("Only application/json content type supported!")

def predict_fn(input_object, pipeline_model):
    """Make predictions on processed input data"""
    predictions = pipeline_model.predict(input_object)
    pred_probs = pipeline_model.predict_proba(input_object)
    
    prediction_object = pd.DataFrame(
        {
            "prediction": predictions.tolist(),
            "pred_prob_class0": pred_probs[:, 0].tolist(),
            "pred_prob_class1": pred_probs[:, 1].tolist()
        }
    )
    
    return prediction_object

def output_fn(prediction_object, request_content_type):
    """Post process the predictions and return as json"""
    return_object = prediction_object.to_json(orient="records", lines=True)
    
    return return_object

Writing serve.py


In [24]:
%%writefile requirements.txt
pandas
numpy

Overwriting requirements.txt


## Real Time Endpoint Deployment

In [25]:
# Create the deployment
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role, Session

session = Session()
bucket = session.default_bucket()

training_job_name = "rfc-pipeline-tuner-240328-1134-010-ae30df28" 
model_artifact = f"s3://{bucket}/{training_job_name}/output/model.tar.gz"
endpoint_name = "anomanly-rfc-pipeline-real-time"

model = SKLearnModel(
    name=endpoint_name,
    framework_version="1.0-1",
    entry_point="serve.py",
    dependencies=["requirements.txt"],
    model_data=model_artifact,
    role=get_execution_role(),
)

In [27]:

# Deploy 
predictor = model.deploy(instance_type="ml.m5.large", initial_instance_count=1)


INFO:sagemaker:Creating model with name: anomanly-rfc-pipeline-real-time
INFO:sagemaker:Creating endpoint-config with name anomanly-rfc-pipeline-real-time-2024-03-28-12-05-29-794
INFO:sagemaker:Creating endpoint with name anomanly-rfc-pipeline-real-time-2024-03-28-12-05-29-794


------!

In [28]:
endpoint_name = predictor.endpoint_name
print("Endpoint name:")
print(f"{endpoint_name}")

Endpoint name:
anomanly-rfc-pipeline-real-time-2024-03-28-12-05-29-794


# Invoke the model

In [32]:
# Load some data that we want to make predictions on
import pandas as pd
test_df = pd.read_csv("s3://sagemaker-eu-north-1-730335315002/sagemaker/anomaly/test.csv")

X_test = test_df.drop("y", axis=1)
y_test = test_df["y"]

# Get two rows to make predictions on
X_pred = X_test.head(2).to_json(orient="records", lines=True)

In [33]:
# Submit to the endpoint
import boto3

sm_runtime = boto3.client("sagemaker-runtime")

response = sm_runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                      Body=X_pred, 
                                      ContentType="application/json", 
                                      Accept="application/json")

In [34]:
# Decode the response from the endpoint
response_body = response['Body']
response_str = response_body.read().decode('utf-8')
response_df = pd.read_json(response_str, lines=True)

response_df

/tmp/ipykernel_1575/2779657148.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  response_df = pd.read_json(response_str, lines=True)


,prediction,pred_prob_class0,pred_prob_class1
0,0,0.998784,0.001216
1,0,0.998063,0.001937
